# 75 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-12-01 06:25:04.694665: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 06:25:04.743490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 06:25:04.743547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 06:25:04.744859: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 06:25:04.752870: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 06:25:04.755959: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
no_of_patients=75

In [3]:
train=sorted(glob('files4/*.edf'))
train=train[:no_of_patients]
train

['files4/S001R06.edf',
 'files4/S002R06.edf',
 'files4/S003R06.edf',
 'files4/S004R06.edf',
 'files4/S005R06.edf',
 'files4/S006R06.edf',
 'files4/S007R06.edf',
 'files4/S008R06.edf',
 'files4/S009R06.edf',
 'files4/S010R06.edf',
 'files4/S011R06.edf',
 'files4/S012R06.edf',
 'files4/S013R06.edf',
 'files4/S014R06.edf',
 'files4/S015R06.edf',
 'files4/S016R06.edf',
 'files4/S017R06.edf',
 'files4/S018R06.edf',
 'files4/S019R06.edf',
 'files4/S020R06.edf',
 'files4/S021R06.edf',
 'files4/S022R06.edf',
 'files4/S023R06.edf',
 'files4/S024R06.edf',
 'files4/S025R06.edf',
 'files4/S026R06.edf',
 'files4/S027R06.edf',
 'files4/S028R06.edf',
 'files4/S029R06.edf',
 'files4/S030R06.edf',
 'files4/S031R06.edf',
 'files4/S032R06.edf',
 'files4/S033R06.edf',
 'files4/S034R06.edf',
 'files4/S035R06.edf',
 'files4/S036R06.edf',
 'files4/S037R06.edf',
 'files4/S038R06.edf',
 'files4/S039R06.edf',
 'files4/S040R06.edf',
 'files4/S041R06.edf',
 'files4/S042R06.edf',
 'files4/S043R06.edf',
 'files4/S0

In [4]:
train_split=0.8

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(900000, 112425, 900000, 112425)

In [10]:
print(xtest.iloc[:,-1].values)

[-5.8e-05 -3.7e-05 -1.0e-06 ... -8.8e-05 -3.7e-05  5.6e-05]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000053,0.000060,0.000035,0.000044,0.000049,0.000032,0.000064,0.000115,0.000076,0.000070,...,0.000020,0.000121,0.000117,0.000060,0.000022,0.000036,0.000135,0.000116,0.000082,0.000140
899996,0.000035,0.000046,0.000006,0.000023,0.000027,-0.000001,0.000035,0.000067,0.000067,0.000043,...,-0.000052,0.000027,0.000048,0.000016,-0.000032,-0.000023,0.000050,0.000045,0.000020,0.000031
899997,0.000031,0.000011,-0.000019,-0.000008,0.000003,-0.000027,0.000007,0.000019,0.000022,0.000012,...,-0.000039,0.000022,0.000042,0.000023,-0.000008,0.000002,0.000035,0.000048,0.000055,0.000033
899998,0.000011,0.000017,0.000021,-0.000007,0.000022,0.000058,0.000063,0.000008,0.000050,0.000035,...,-0.000041,0.000021,0.000048,0.000049,0.000023,0.000002,0.000028,0.000028,0.000037,0.000026


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_41757/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_41757/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_41757/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000053,0.000060,0.000035,0.000044,0.000049,0.000032,0.000064,0.000115,0.000076,0.000070,...,0.000020,0.000121,0.000117,0.000060,0.000022,0.000036,0.000135,0.000116,0.000082,0.000140
899996,0.000035,0.000046,0.000006,0.000023,0.000027,-0.000001,0.000035,0.000067,0.000067,0.000043,...,-0.000052,0.000027,0.000048,0.000016,-0.000032,-0.000023,0.000050,0.000045,0.000020,0.000031
899997,0.000031,0.000011,-0.000019,-0.000008,0.000003,-0.000027,0.000007,0.000019,0.000022,0.000012,...,-0.000039,0.000022,0.000042,0.000023,-0.000008,0.000002,0.000035,0.000048,0.000055,0.000033
899998,0.000011,0.000017,0.000021,-0.000007,0.000022,0.000058,0.000063,0.000008,0.000050,0.000035,...,-0.000041,0.000021,0.000048,0.000049,0.000023,0.000002,0.000028,0.000028,0.000037,0.000026


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:4.07270
[1]	validation_0-mlogloss:3.93581
[2]	validation_0-mlogloss:3.85327
[3]	validation_0-mlogloss:3.79386
[4]	validation_0-mlogloss:3.74048
[5]	validation_0-mlogloss:3.69728
[6]	validation_0-mlogloss:3.66384
[7]	validation_0-mlogloss:3.63004
[8]	validation_0-mlogloss:3.60399
[9]	validation_0-mlogloss:3.58090
[10]	validation_0-mlogloss:3.55989
[11]	validation_0-mlogloss:3.53880
[12]	validation_0-mlogloss:3.52509
[13]	validation_0-mlogloss:3.51159
[14]	validation_0-mlogloss:3.49451
[15]	validation_0-mlogloss:3.48406
[16]	validation_0-mlogloss:3.47314
[17]	validation_0-mlogloss:3.45942
[18]	validation_0-mlogloss:3.44737
[19]	validation_0-mlogloss:3.43632
[20]	validation_0-mlogloss:3.42647
[21]	validation_0-mlogloss:3.41868
[22]	validation_0-mlogloss:3.40933
[23]	validation_0-mlogloss:3.40177
[24]	validation_0-mlogloss:3.39491
[25]	validation_0-mlogloss:3.38508
[26]	validation_0-mlogloss:3.37627
[27]	validation_0-mlogloss:3.36858
[28]	validation_0-mlogloss:3.3

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
28125/28125 [==============================] - 24s 841us/step - loss: 2.7382 - val_loss: 3.1125
Epoch 2/10
28125/28125 [==============================] - 23s 821us/step - loss: 2.1957 - val_loss: 3.0850
Epoch 3/10
28125/28125 [==============================] - 24s 839us/step - loss: 2.0590 - val_loss: 3.0434
Epoch 4/10
28125/28125 [==============================] - 23s 800us/step - loss: 2.0088 - val_loss: 3.0299
Epoch 5/10
28125/28125 [==============================] - 23s 824us/step - loss: 1.9786 - val_loss: 3.0936
Epoch 6/10
28125/28125 [==============================] - 24s 836us/step - loss: 1.9555 - val_loss: 3.0588
Epoch 7/10
28125/28125 [==============================] - 24s 870us/step - loss: 1.9370 - val_loss: 3.0399
Epoch 8/10
28125/28125 [==============================] - 25s 882us/step - loss: 1.9241 - val_loss: 3.1220
Epoch 9/10
28125/28125 [==============================] - 24s 864us/step - loss: 1.9105 - val_loss: 3.2073
Epoch 10/10
28125/28125 [============

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

3514/3514 [==============================] - 1s 359us/step
              precision    recall  f1-score   support

           0       0.12      0.07      0.09      1499
           1       0.06      0.05      0.06      1499
           2       0.30      0.13      0.18      1499
           3       0.72      0.57      0.63      1499
           4       0.75      0.69      0.72      1499
           5       0.33      0.26      0.29      1499
           6       0.40      0.35      0.37      1499
           7       0.56      0.45      0.50      1499
           8       0.50      0.62      0.55      1499
           9       0.76      0.27      0.40      1499
          10       0.17      0.11      0.13      1499
          11       0.15      0.34      0.20      1499
          12       0.24      0.08      0.11      1499
          13       0.09      0.04      0.06      1499
          14       0.15      0.05      0.08      1499
          15       0.29      0.19      0.23      1499
          16       0.1

## 0-16

In [17]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [18]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_41757/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_41757/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_41757/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,0.000073,0.000073,0.000091,0.000116,0.000101,0.000107,0.000062,0.000035,0.000058,0.000058,...,0.000036,0.000048,0.000046,0.000065,0.000039,0.000073,0.000029,-0.000004,0.000059,-0.000092
1,0.000063,0.000064,0.000078,0.000107,0.000086,0.000087,0.000044,0.000036,0.000061,0.000058,...,0.000043,0.000062,0.000056,0.000068,0.000043,0.000088,0.000032,0.000004,0.000069,-0.000093
2,0.000082,0.000074,0.000082,0.000100,0.000083,0.000081,0.000048,0.000049,0.000067,0.000058,...,0.000041,0.000049,0.000040,0.000043,0.000025,0.000079,0.000025,-0.000003,0.000065,-0.000102
3,0.000063,0.000042,0.000050,0.000065,0.000059,0.000061,0.000032,0.000024,0.000035,0.000023,...,0.000034,0.000040,0.000029,0.000033,0.000014,0.000069,0.000013,-0.000016,0.000050,-0.000107
4,0.000051,0.000039,0.000052,0.000081,0.000069,0.000073,0.000039,0.000018,0.000035,0.000030,...,0.000034,0.000056,0.000049,0.000056,0.000025,0.000075,0.000015,-0.000013,0.000044,-0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000053,0.000060,0.000035,0.000044,0.000049,0.000032,0.000064,0.000115,0.000076,0.000070,...,0.000020,0.000121,0.000117,0.000060,0.000022,0.000036,0.000135,0.000116,0.000082,0.000140
899996,0.000035,0.000046,0.000006,0.000023,0.000027,-0.000001,0.000035,0.000067,0.000067,0.000043,...,-0.000052,0.000027,0.000048,0.000016,-0.000032,-0.000023,0.000050,0.000045,0.000020,0.000031
899997,0.000031,0.000011,-0.000019,-0.000008,0.000003,-0.000027,0.000007,0.000019,0.000022,0.000012,...,-0.000039,0.000022,0.000042,0.000023,-0.000008,0.000002,0.000035,0.000048,0.000055,0.000033
899998,0.000011,0.000017,0.000021,-0.000007,0.000022,0.000058,0.000063,0.000008,0.000050,0.000035,...,-0.000041,0.000021,0.000048,0.000049,0.000023,0.000002,0.000028,0.000028,0.000037,0.000026


In [19]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [20]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.97688
[1]	validation_0-mlogloss:3.82565
[2]	validation_0-mlogloss:3.71997
[3]	validation_0-mlogloss:3.63783
[4]	validation_0-mlogloss:3.56949
[5]	validation_0-mlogloss:3.51264
[6]	validation_0-mlogloss:3.46750
[7]	validation_0-mlogloss:3.42456
[8]	validation_0-mlogloss:3.38510
[9]	validation_0-mlogloss:3.35379
[10]	validation_0-mlogloss:3.32325
[11]	validation_0-mlogloss:3.29447
[12]	validation_0-mlogloss:3.26926
[13]	validation_0-mlogloss:3.24776
[14]	validation_0-mlogloss:3.22389
[15]	validation_0-mlogloss:3.20452
[16]	validation_0-mlogloss:3.18528
[17]	validation_0-mlogloss:3.16811
[18]	validation_0-mlogloss:3.15288
[19]	validation_0-mlogloss:3.13718
[20]	validation_0-mlogloss:3.12502
[21]	validation_0-mlogloss:3.11146
[22]	validation_0-mlogloss:3.09980
[23]	validation_0-mlogloss:3.08305
[24]	validation_0-mlogloss:3.07079
[25]	validation_0-mlogloss:3.05778
[26]	validation_0-mlogloss:3.04830
[27]	validation_0-mlogloss:3.03679
[28]	validation_0-mlogloss:3.0

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
28125/28125 [==============================] - 25s 878us/step - loss: 2.0846 - val_loss: 2.9505
Epoch 2/10
28125/28125 [==============================] - 24s 862us/step - loss: 1.4164 - val_loss: 3.0817
Epoch 3/10
28125/28125 [==============================] - 24s 865us/step - loss: 1.2646 - val_loss: 3.2629
Epoch 4/10
28125/28125 [==============================] - 25s 879us/step - loss: 1.1796 - val_loss: 3.1828
Epoch 5/10
28125/28125 [==============================] - 24s 863us/step - loss: 1.1058 - val_loss: 3.2632
Epoch 6/10
28125/28125 [==============================] - 25s 892us/step - loss: 1.0445 - val_loss: 3.3203
Epoch 7/10
28125/28125 [==============================] - 24s 870us/step - loss: 1.0056 - val_loss: 3.2761
Epoch 8/10
28125/28125 [==============================] - 25s 879us/step - loss: 0.9806 - val_loss: 3.2759
Epoch 9/10
28125/28125 [==============================] - 25s 885us/step - loss: 0.9608 - val_loss: 3.5193
Epoch 10/10
28125/28125 [============

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

3514/3514 [==============================] - 1s 360us/step
              precision    recall  f1-score   support

           0       0.30      0.34      0.32      1499
           1       0.21      0.10      0.13      1499
           2       0.38      0.14      0.21      1499
           3       0.62      0.60      0.61      1499
           4       0.80      0.84      0.82      1499
           5       0.52      0.68      0.59      1499
           6       0.72      0.78      0.75      1499
           7       0.67      0.58      0.62      1499
           8       0.52      0.69      0.59      1499
           9       0.58      0.59      0.58      1499
          10       0.49      0.35      0.41      1499
          11       0.50      0.33      0.40      1499
          12       0.44      0.44      0.44      1499
          13       0.20      0.23      0.21      1499
          14       0.26      0.08      0.12      1499
          15       0.74      0.25      0.38      1499
          16       0.1

## 0-32

In [21]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [22]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_41757/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_41757/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_41757/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [23]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [24]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.78055
[1]	validation_0-mlogloss:3.57016
[2]	validation_0-mlogloss:3.42238
[3]	validation_0-mlogloss:3.30021
[4]	validation_0-mlogloss:3.20967
[5]	validation_0-mlogloss:3.13224
[6]	validation_0-mlogloss:3.06683
[7]	validation_0-mlogloss:3.01234
[8]	validation_0-mlogloss:2.96362
[9]	validation_0-mlogloss:2.91947
[10]	validation_0-mlogloss:2.87832
[11]	validation_0-mlogloss:2.83500
[12]	validation_0-mlogloss:2.80239
[13]	validation_0-mlogloss:2.77259
[14]	validation_0-mlogloss:2.74162
[15]	validation_0-mlogloss:2.71366
[16]	validation_0-mlogloss:2.68637
[17]	validation_0-mlogloss:2.66215
[18]	validation_0-mlogloss:2.63690
[19]	validation_0-mlogloss:2.61959
[20]	validation_0-mlogloss:2.59564
[21]	validation_0-mlogloss:2.57510
[22]	validation_0-mlogloss:2.55419
[23]	validation_0-mlogloss:2.53688
[24]	validation_0-mlogloss:2.51913
[25]	validation_0-mlogloss:2.50247
[26]	validation_0-mlogloss:2.48338
[27]	validation_0-mlogloss:2.46903
[28]	validation_0-mlogloss:2.4

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
28125/28125 [==============================] - 27s 955us/step - loss: 1.4301 - val_loss: 2.7131
Epoch 2/10
28125/28125 [==============================] - 26s 929us/step - loss: 0.6283 - val_loss: 2.5845
Epoch 3/10
28125/28125 [==============================] - 27s 944us/step - loss: 0.4776 - val_loss: 2.5206
Epoch 4/10
28125/28125 [==============================] - 29s 1ms/step - loss: 0.4093 - val_loss: 2.6662
Epoch 5/10
28125/28125 [==============================] - 27s 960us/step - loss: 0.3673 - val_loss: 2.7442
Epoch 6/10
28125/28125 [==============================] - 27s 950us/step - loss: 0.3391 - val_loss: 2.6514
Epoch 7/10
28125/28125 [==============================] - 26s 938us/step - loss: 0.3166 - val_loss: 2.7831
Epoch 8/10
28125/28125 [==============================] - 27s 947us/step - loss: 0.2994 - val_loss: 2.6584
Epoch 9/10
28125/28125 [==============================] - 26s 937us/step - loss: 0.2848 - val_loss: 2.7271
Epoch 10/10
28125/28125 [==============

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

3514/3514 [==============================] - 1s 383us/step
              precision    recall  f1-score   support

           0       0.74      0.57      0.64      1499
           1       0.31      0.14      0.19      1499
           2       0.62      0.74      0.67      1499
           3       0.72      0.46      0.56      1499
           4       0.90      0.93      0.92      1499
           5       0.85      0.88      0.86      1499
           6       0.96      0.72      0.82      1499
           7       0.82      0.53      0.64      1499
           8       0.68      0.85      0.75      1499
           9       0.71      0.73      0.72      1499
          10       0.61      0.94      0.74      1499
          11       0.81      0.95      0.87      1499
          12       0.80      0.76      0.78      1499
          13       0.57      0.53      0.55      1499
          14       0.52      0.66      0.58      1499
          15       0.76      0.77      0.77      1499
          16       0.6

## 0-64

In [13]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [14]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [15]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.54983
[1]	validation_0-mlogloss:3.25889
[2]	validation_0-mlogloss:3.06115
[3]	validation_0-mlogloss:2.92295
[4]	validation_0-mlogloss:2.80457
[5]	validation_0-mlogloss:2.70221
[6]	validation_0-mlogloss:2.62148
[7]	validation_0-mlogloss:2.54302
[8]	validation_0-mlogloss:2.47749
[9]	validation_0-mlogloss:2.41887
[10]	validation_0-mlogloss:2.36722
[11]	validation_0-mlogloss:2.32322
[12]	validation_0-mlogloss:2.27935
[13]	validation_0-mlogloss:2.23690
[14]	validation_0-mlogloss:2.19956
[15]	validation_0-mlogloss:2.16809
[16]	validation_0-mlogloss:2.13115
[17]	validation_0-mlogloss:2.10099
[18]	validation_0-mlogloss:2.06963
[19]	validation_0-mlogloss:2.04097
[20]	validation_0-mlogloss:2.01682
[21]	validation_0-mlogloss:1.99170
[22]	validation_0-mlogloss:1.96983
[23]	validation_0-mlogloss:1.94550
[24]	validation_0-mlogloss:1.92654
[25]	validation_0-mlogloss:1.90653
[26]	validation_0-mlogloss:1.88677
[27]	validation_0-mlogloss:1.87071
[28]	validation_0-mlogloss:1.8

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
28125/28125 [==============================] - 29s 1ms/step - loss: 1.0420 - val_loss: 2.6500
Epoch 2/10
28125/28125 [==============================] - 28s 999us/step - loss: 0.4017 - val_loss: 2.5875
Epoch 3/10
28125/28125 [==============================] - 28s 996us/step - loss: 0.2887 - val_loss: 2.5339
Epoch 4/10
28125/28125 [==============================] - 28s 1ms/step - loss: 0.2335 - val_loss: 2.4446
Epoch 5/10
28125/28125 [==============================] - 28s 1ms/step - loss: 0.1976 - val_loss: 2.5517
Epoch 6/10
28125/28125 [==============================] - 28s 1ms/step - loss: 0.1750 - val_loss: 2.6834
Epoch 7/10
28125/28125 [==============================] - 28s 1ms/step - loss: 0.1549 - val_loss: 2.3723
Epoch 8/10
28125/28125 [==============================] - 23s 806us/step - loss: 0.1412 - val_loss: 2.5446
Epoch 9/10
28125/28125 [==============================] - 22s 766us/step - loss: 0.1295 - val_loss: 2.5495
Epoch 10/10
28125/28125 [======================

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

3514/3514 [==============================] - 1s 317us/step
              precision    recall  f1-score   support

           0       0.93      0.66      0.77      1499
           1       0.70      0.40      0.51      1499
           2       0.83      0.73      0.78      1499
           3       0.96      0.80      0.87      1499
           4       0.95      0.99      0.97      1499
           5       0.96      0.98      0.97      1499
           6       0.95      0.83      0.89      1499
           7       0.81      0.67      0.73      1499
           8       0.78      0.91      0.84      1499
           9       0.96      0.89      0.92      1499
          10       0.68      0.77      0.72      1499
          11       0.97      0.99      0.98      1499
          12       0.89      0.85      0.87      1499
          13       0.74      0.82      0.78      1499
          14       0.63      0.88      0.73      1499
          15       0.96      0.96      0.96      1499
          16       0.9